<a href="https://colab.research.google.com/github/jasperSha/cloud_music/blob/main/feature_collection/mel_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud projects list

In [2]:
import os

os.makedirs('song_images', exist_ok=True)

os.makedirs('song_flacs', exist_ok=True)

In [ ]:
# load audio every N batches
# have to manually enter (only have to change tail input to starting index each time, head just adds to it)
start = 0
end = 10

'''
TODO: for each batch, create csv of just the id's of song images created, use that for reference(instead of loading the pngs)
'''
# load image records data
!gsutil -m cp gs://deepclustermusic/song_images/*.csv .

!gsutil ls gs://deepclustermusic/flac_files/spotify_yt_data/*.flac | tail -n +0 | head -n 10 | gsutil -m cp -n -I './song_flacs'


In [ ]:
!pip install fastaudio

In [ ]:
import glob

import numpy as np
import pandas as pd

import librosa
import librosa.feature

import torchaudio
import torch

In [ ]:
mel_config = dict(
    sample_rate=44100,
    hop_length=308,
    win_length=1024, # 1024 combined with n_fft=4096 best; 2205 -> 50ms, 4410 -> 100ms
    n_mels=224,
    n_fft=4096,
    f_max=22500, # get the cymbals, hi-hats, etc frequencies >20k
)

In [ ]:
def normalize_sample(waveform):
  '''
  @param waveform: nd array
  '''
  return (waveform - waveform.mean()) / waveform.std()

In [ ]:
flacs = list(glob.glob('/content/song_flacs/*.flac'))

fin = list(glob.glob('*.csv'))
finished_df = pd.concat((pd.read_csv(f) for f in finished))

In [ ]:
songs = []
for fname in flacs:
  song_id = fname.split('/')[-1].split('.')[0]

  if song_id in finished_df['id'].values:
    print('already finished this one')
    continue

  signal, sr = torchaudio.load(fname)
  channel = signal.ndim

  # convert any stereo to mono
  if channel == 2:
    signal = torch.mean(signal, dim=0, keepdim=True)

  if signal is None:
    continue

  # get duration in seconds
  num_elements = torch.numel(signal)
  duration = int(num_elements/sr)

  if duration < 120 or duration > 480:
    continue

